In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Klaim.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/sample_submission.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Polis.csv


# DATA FOUNDATION

In [2]:
# ============================================================
# STAGE 1 v6 — STRUCTURAL FOUNDATION (PRO VERSION)
# Payment/Service switch • Full month continuity • No NaN severity
# Exposure sanity check • Delay modeling ready
# ============================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"

klaim = pd.read_csv(BASE_PATH + "Data_Klaim.csv")
polis = pd.read_csv(BASE_PATH + "Data_Polis.csv")

# ============================================================
# CLEAN COLUMNS
# ============================================================

def clean_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("/", "_", regex=False)
        .str.replace("-", "_", regex=False)
    )
    return df

klaim = clean_columns(klaim)
polis = clean_columns(polis)

klaim = klaim.drop_duplicates().reset_index(drop=True)
polis = polis.drop_duplicates().reset_index(drop=True)

# ============================================================
# DATE PARSING
# ============================================================

for col in klaim.columns:
    if "tanggal" in col:
        klaim[col] = pd.to_datetime(klaim[col], errors="coerce")

for col in polis.columns:
    if "tanggal" in col:
        polis[col] = pd.to_datetime(polis[col], errors="coerce")

# ============================================================
# BASIC CLEAN
# ============================================================

klaim = klaim.dropna(subset=["nomor_polis"]).copy()
klaim["nominal_klaim_yang_disetujui"] = klaim["nominal_klaim_yang_disetujui"].fillna(0)

df = klaim.merge(polis, on="nomor_polis", how="left")

# ============================================================
# MONTH AXIS SWITCH
# ============================================================

MONTH_TYPE = "PAYMENT"   # SERVICE or PAYMENT

if MONTH_TYPE == "SERVICE":
    month_col = "tanggal_pasien_masuk_rs"
elif MONTH_TYPE == "PAYMENT":
    month_col = "tanggal_pembayaran_klaim"
else:
    raise ValueError("MONTH_TYPE must be SERVICE or PAYMENT")

df = df.dropna(subset=[month_col]).copy()
df["year_month"] = df[month_col].dt.to_period("M")

# ============================================================
# DELAY FEATURE (IMPORTANT FOR LATER STAGES)
# ============================================================

if "tanggal_pasien_masuk_rs" in df.columns and \
   "tanggal_pembayaran_klaim" in df.columns:
    
    df["delay_days"] = (
        df["tanggal_pembayaran_klaim"] -
        df["tanggal_pasien_masuk_rs"]
    ).dt.days

# ============================================================
# MONTHLY AGGREGATION
# ============================================================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .sort_index()
)

# ============================================================
# ENSURE FULL MONTH CONTINUITY (VERY IMPORTANT)
# ============================================================

full_index = pd.period_range(
    monthly.index.min(),
    monthly.index.max(),
    freq="M"
)

monthly = monthly.reindex(full_index)

monthly["frequency"] = monthly["frequency"].fillna(0)
monthly["total_claim"] = monthly["total_claim"].fillna(0)

# ============================================================
# SAFE SEVERITY (NO NaN)
# ============================================================

monthly["severity"] = np.where(
    monthly["frequency"] > 0,
    monthly["total_claim"] / monthly["frequency"],
    0
)

monthly = monthly.reset_index().rename(columns={"index":"year_month"})

# ============================================================
# EXPOSURE STABILITY CHECK
# ============================================================

freq_cv = monthly["frequency"].std() / (monthly["frequency"].mean() + 1e-9)
total_cv = monthly["total_claim"].std() / (monthly["total_claim"].mean() + 1e-9)

# ============================================================
# LOG DOMAIN FEATURES (FOR STAGE 3+)
# ============================================================

monthly["log_freq"] = np.log1p(monthly["frequency"])
monthly["log_total"] = np.log1p(monthly["total_claim"])

# ============================================================
# INFO OUTPUT
# ============================================================

print("====================================")
print("STAGE 1 v6 READY")
print("Month type:", MONTH_TYPE)
print("Total months:", len(monthly))
print("Frequency CV:", round(freq_cv,3))
print("Total CV:", round(total_cv,3))
print("Date range:", monthly["year_month"].min(), "→", monthly["year_month"].max())
print("====================================")

monthly.head()


STAGE 1 v6 READY
Month type: PAYMENT
Total months: 24
Frequency CV: 0.515
Total CV: 0.568
Date range: 2024-01 → 2025-12


,year_month,frequency,total_claim,severity,log_freq,log_total
0,2024-01,8,1.283162e+08,1.603952e+07,2.197225,18.670008
1,2024-02,92,2.684171e+09,2.917577e+07,4.532599,21.710638
2,2024-03,97,3.809944e+09,3.927778e+07,4.584967,22.060880
3,2024-04,221,9.281203e+09,4.199640e+07,5.402677,22.951257
4,2024-05,233,1.103847e+10,4.737540e+07,5.455321,23.124652


# TIME-SERIES DATASET ENGINEERING

In [3]:
# ============================================================
# STAGE 2 v4 — STRUCTURAL SHARE PANEL (SHORT SERIES SAFE)
# Plan filtering • Full month continuity • Share constraint
# No aggressive lag drop
# ============================================================

import numpy as np
import pandas as pd

# ============================================================
# ENSURE PLAN CODE
# ============================================================

if "plan_code" not in df.columns:
    df["plan_code"] = "UNKNOWN"

df["plan_code"] = df["plan_code"].fillna("UNKNOWN")

# ============================================================
# BUILD GLOBAL MONTHLY (FROM STAGE 1)
# ============================================================

global_monthly = (
    df.groupby("year_month")
      .agg(
          total_global=("nominal_klaim_yang_disetujui","sum"),
          freq_global=("claim_id","count")
      )
      .reset_index()
      .sort_values("year_month")
)

# ============================================================
# PLAN-LEVEL MONTHLY
# ============================================================

seg_monthly = (
    df.groupby(["year_month","plan_code"])
      .agg(
          total_claim=("nominal_klaim_yang_disetujui","sum"),
          frequency=("claim_id","count")
      )
      .reset_index()
)

# ============================================================
# FILTER SMALL PLANS (IMPORTANT)
# ============================================================

plan_volume = (
    seg_monthly.groupby("plan_code")["total_claim"]
    .sum()
    .reset_index()
)

# Keep only plans contributing ≥ 3% total volume
total_all = plan_volume["total_claim"].sum()
plan_volume["share"] = plan_volume["total_claim"] / total_all

valid_plans = plan_volume.loc[
    plan_volume["share"] >= 0.03,
    "plan_code"
]

seg_monthly = seg_monthly[
    seg_monthly["plan_code"].isin(valid_plans)
].copy()

# ============================================================
# ENSURE FULL MONTH CONTINUITY PER PLAN
# ============================================================

full_months = pd.period_range(
    df["year_month"].min(),
    df["year_month"].max(),
    freq="M"
)

panel = []

for plan in seg_monthly["plan_code"].unique():
    
    temp = seg_monthly[seg_monthly["plan_code"] == plan]
    temp = temp.set_index("year_month").reindex(full_months)
    
    temp["plan_code"] = plan
    temp["total_claim"] = temp["total_claim"].fillna(0)
    temp["frequency"] = temp["frequency"].fillna(0)
    
    panel.append(temp.reset_index())

seg_monthly = pd.concat(panel, ignore_index=True)
seg_monthly = seg_monthly.rename(columns={"index":"year_month"})

# ============================================================
# MERGE GLOBAL FOR SHARE
# ============================================================

seg_monthly = seg_monthly.merge(
    global_monthly,
    on="year_month",
    how="left"
)

# ============================================================
# SHARE CALCULATION
# ============================================================

seg_monthly["share_total"] = np.where(
    seg_monthly["total_global"] > 0,
    seg_monthly["total_claim"] / seg_monthly["total_global"],
    0
)

# ============================================================
# SHARE SMOOTHING (LIGHT, SAFE)
# ============================================================

seg_monthly["share_smooth"] = (
    seg_monthly.groupby("plan_code")["share_total"]
    .transform(lambda x: x.rolling(2, min_periods=1).mean())
)

# ============================================================
# RE-NORMALIZE SHARE (SUM TO 1 PER MONTH)
# ============================================================

seg_monthly["share_norm"] = (
    seg_monthly["share_smooth"] /
    seg_monthly.groupby("year_month")["share_smooth"].transform("sum")
)

# ============================================================
# OPTIONAL LOG FEATURES (FOR ADVANCED STAGE)
# ============================================================

seg_monthly["log_total"] = np.log1p(seg_monthly["total_claim"])
seg_monthly["log_freq"]  = np.log1p(seg_monthly["frequency"])

# ============================================================
# FINAL CHECK
# ============================================================

print("====================================")
print("STAGE 2 v4 READY")
print("Plans kept:", seg_monthly["plan_code"].nunique())
print("Months:", seg_monthly["year_month"].nunique())
print("Min months per plan:",
      seg_monthly.groupby("plan_code")["year_month"].nunique().min())
print("Share sum check (last month):",
      round(
          seg_monthly.groupby("year_month")["share_norm"]
          .sum()
          .iloc[-1],
          4
      ))
print("====================================")

seg_monthly.head()


STAGE 2 v4 READY
Plans kept: 3
Months: 24
Min months per plan: 24
Share sum check (last month): 1.0


,year_month,plan_code,total_claim,frequency,total_global,freq_global,share_total,share_smooth,share_norm,log_total,log_freq
0,2024-01,M-002,7.378536e+07,6.0,1.283162e+08,8,0.575028,0.575028,0.575028,18.116671,1.945910
1,2024-02,M-002,2.195347e+09,69.0,2.684171e+09,92,0.817886,0.696457,0.696457,21.509606,4.248495
2,2024-03,M-002,1.921474e+09,68.0,3.809944e+09,97,0.504331,0.661109,0.661109,21.376358,4.234107
3,2024-04,M-002,6.031108e+09,143.0,9.281203e+09,221,0.649820,0.577075,0.577075,22.520197,4.969813
4,2024-05,M-002,7.416628e+09,158.0,1.103847e+10,233,0.671889,0.660854,0.660854,22.726990,5.068904


# MODEL DEVELOPMENT

In [4]:
# ============================================================
# STAGE 3 v8 — MONITOR VERSION
# Exact Kaggle logic + readable output
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

# ============================================================
# KAGGLE MAPE (NO *100)
# ============================================================

def mape_raw(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

# ============================================================
# BUILD SERVICE MONTHLY
# ============================================================

df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .sort_index()
)

monthly["severity"] = np.where(
    monthly["frequency"] > 0,
    monthly["total_claim"] / monthly["frequency"],
    0
)

monthly["log_freq"]  = np.log1p(monthly["frequency"])
monthly["log_total"] = np.log1p(monthly["total_claim"])

monthly = monthly.reset_index()

print("Total SERVICE months:", len(monthly))

# ============================================================
# SINGLE HOLDOUT (MATCH LEADERBOARD STRUCTURE)
# ============================================================

H = 4
train = monthly.iloc[:-H]
valid = monthly.iloc[-H:]

# ============================================================
# FIT MODELS
# ============================================================

model_total = ExponentialSmoothing(
    train["log_total"],
    trend="add",
    damped_trend=True
).fit()

model_freq = ExponentialSmoothing(
    train["log_freq"],
    trend="add",
    damped_trend=True
).fit()

# ============================================================
# FORECAST
# ============================================================

total_fc = np.expm1(model_total.forecast(H))
freq_fc  = np.expm1(model_freq.forecast(H))

# Shrink stabilizer
shrink = 0.97
for i in range(H):
    total_fc.iloc[i] *= shrink**i
    freq_fc.iloc[i]  *= shrink**i

sev_fc = np.where(freq_fc > 0, total_fc / freq_fc, 0)

# ============================================================
# COMPUTE MAPE (DISPLAY IN %)
# ============================================================

mape_freq = mape_raw(valid["frequency"], freq_fc) * 100
mape_sev  = mape_raw(valid["severity"], sev_fc) * 100
mape_tot  = mape_raw(valid["total_claim"], total_fc) * 100

final_score = (mape_freq + mape_sev + mape_tot) / 3

# ============================================================
# PRINT MONITOR FORMAT
# ============================================================

print("\n==============================")
print("MAPE Frequency :", round(mape_freq,4))
print("MAPE Severity  :", round(mape_sev,4))
print("MAPE Total     :", round(mape_tot,4))
print("Estimated Score:", round(final_score,4))
print("==============================")


Total SERVICE months: 19

MAPE Frequency : 9.2117
MAPE Severity  : 2.4429
MAPE Total     : 8.6838
Estimated Score: 6.7795


# TOTAL CLAIM OPTIMIZATION & VALIDATION, OPTUNA

In [5]:
# ============================================================
# STAGE 4 v6 — SMALL SERIES OPTIMIZED
# ETS TOTAL + ETS FREQ
# Derive SEVERITY
# Rolling CV + Shrink
# ============================================================

!pip install -q optuna

import optuna
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

# ============================================================
# METRIC (KAGGLE EXACT)
# ============================================================

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

def kaggle_score(freq_t, freq_p, sev_t, sev_p, total_t, total_p):
    return (
        mape(freq_t, freq_p) +
        mape(sev_t, sev_p) +
        mape(total_t, total_p)
    ) / 3

# ============================================================
# BUILD MONTHLY
# ============================================================

df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .sort_index()
)

monthly["severity"] = np.where(
    monthly["frequency"] > 0,
    monthly["total_claim"] / monthly["frequency"],
    0
)

monthly["log_total"] = np.log1p(monthly["total_claim"])
monthly["log_freq"]  = np.log1p(monthly["frequency"])

monthly = monthly.reset_index()

# ============================================================
# SETTINGS
# ============================================================

H = 4
MIN_TRAIN = 12

# ============================================================
# OBJECTIVE
# ============================================================

def objective(trial):

    alpha_t = trial.suggest_float("alpha_total", 0.1, 0.9)
    beta_t  = trial.suggest_float("beta_total", 0.01, 0.3)
    phi_t   = trial.suggest_float("phi_total", 0.7, 0.99)

    alpha_f = trial.suggest_float("alpha_freq", 0.1, 0.9)
    beta_f  = trial.suggest_float("beta_freq", 0.01, 0.3)
    phi_f   = trial.suggest_float("phi_freq", 0.7, 0.99)

    shrink  = trial.suggest_float("shrink", 0.94, 0.99)

    scores = []

    for split_point in range(MIN_TRAIN, len(monthly) - H + 1):

        train_df = monthly.iloc[:split_point]
        valid_df = monthly.iloc[split_point:split_point + H]

        try:
            model_total = ExponentialSmoothing(
                train_df["log_total"],
                trend="add",
                damped_trend=True
            ).fit(
                smoothing_level=alpha_t,
                smoothing_trend=beta_t,
                damping_trend=phi_t,
                optimized=False
            )

            model_freq = ExponentialSmoothing(
                train_df["log_freq"],
                trend="add",
                damped_trend=True
            ).fit(
                smoothing_level=alpha_f,
                smoothing_trend=beta_f,
                damping_trend=phi_f,
                optimized=False
            )

        except:
            continue

        total_fc = np.expm1(model_total.forecast(H))
        freq_fc  = np.expm1(model_freq.forecast(H))

        # ===== SHRINK STABILIZER =====
        for i in range(H):
            total_fc.iloc[i] *= shrink**i
            freq_fc.iloc[i]  *= shrink**i

        sev_fc = np.where(freq_fc > 0, total_fc / freq_fc, 0)

        score = kaggle_score(
            valid_df["frequency"], freq_fc,
            valid_df["severity"], sev_fc,
            valid_df["total_claim"], total_fc
        )

        scores.append(score)

    if len(scores) == 0:
        return 1.0  # penalize broken trials

    return np.mean(scores)

# ============================================================
# RUN OPTUNA
# ============================================================

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)

print("\nBest Params:")
print(study.best_params)
print("Rolling CV Score:", round(study.best_value*100,4), "%")


[I 2026-02-19 07:48:58,004] A new study created in memory with name: no-name-4298c769-529f-4d1d-a20d-4d91202ac8af
[I 2026-02-19 07:48:58,033] Trial 0 finished with value: 0.09737841305389651 and parameters: {'alpha_total': 0.14152169677764262, 'beta_total': 0.0817492537793029, 'phi_total': 0.7672056800720738, 'alpha_freq': 0.13401166598896613, 'beta_freq': 0.10987946552974695, 'phi_freq': 0.8247247326212918, 'shrink': 0.9542489001859888}. Best is trial 0 with value: 0.09737841305389651.
[I 2026-02-19 07:48:58,058] Trial 1 finished with value: 0.14054396272564318 and parameters: {'alpha_total': 0.5839506952608648, 'beta_total': 0.11728986519677002, 'phi_total': 0.8109580382714967, 'alpha_freq': 0.21300676143546202, 'beta_freq': 0.04072281547221891, 'phi_freq': 0.8543242018462874, 'shrink': 0.9618467061770575}. Best is trial 0 with value: 0.09737841305389651.
[I 2026-02-19 07:48:58,086] Trial 2 finished with value: 0.13952287922704149 and parameters: {'alpha_total': 0.6576001281701016, '


Best Params:
{'alpha_total': 0.10062831368902807, 'beta_total': 0.048081069073956186, 'phi_total': 0.8564159526923939, 'alpha_freq': 0.2584678113045896, 'beta_freq': 0.11305871055839573, 'phi_freq': 0.7441807317831034, 'shrink': 0.9832216977617833}
Rolling CV Score: 8.898 %


# TEST PREDICTION & KAGGLE SUBMISSION

In [6]:
# ============================================================
# STAGE 5 FINAL — MATCH STAGE 4 OPTUNA
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"
sample_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")

# ============================================================
# DOMAIN (HARUS SAMA DENGAN STAGE 4)
# ============================================================

USE_PAYMENT = False

if USE_PAYMENT:
    df["year_month"] = df["tanggal_pembayaran_klaim"].dt.to_period("M")
else:
    df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

# ============================================================
# PASTE PARAMS DARI STAGE 4
# ============================================================

BEST = {
    "alpha_total": 0.13,
    "beta_total": 0.09,
    "phi_total": 0.84,
    "alpha_freq": 0.27,
    "beta_freq": 0.06,
    "phi_freq": 0.96,
    "shrink": 0.97
}

# ============================================================
# BUILD MONTHLY
# ============================================================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .sort_index()
)

monthly["severity"] = np.where(
    monthly["frequency"] > 0,
    monthly["total_claim"] / monthly["frequency"],
    0
)

monthly["log_total"] = np.log1p(monthly["total_claim"])
monthly["log_freq"]  = np.log1p(monthly["frequency"])

monthly = monthly.reset_index()

# ============================================================
# FIT FULL HISTORY WITH OPTUNA PARAMS
# ============================================================

model_total = ExponentialSmoothing(
    monthly["log_total"],
    trend="add",
    damped_trend=True
).fit(
    smoothing_level=BEST["alpha_total"],
    smoothing_trend=BEST["beta_total"],
    damping_trend=BEST["phi_total"],
    optimized=False
)

model_freq = ExponentialSmoothing(
    monthly["log_freq"],
    trend="add",
    damped_trend=True
).fit(
    smoothing_level=BEST["alpha_freq"],
    smoothing_trend=BEST["beta_freq"],
    damping_trend=BEST["phi_freq"],
    optimized=False
)

# ============================================================
# FUTURE PERIODS
# ============================================================

sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]

future_periods = (
    pd.PeriodIndex(sample_sub["month_key"], freq="M")
      .unique()
      .sort_values()
)

H = len(future_periods)

# ============================================================
# FORECAST
# ============================================================

total_fc = np.expm1(model_total.forecast(H))
freq_fc  = np.expm1(model_freq.forecast(H))

# SHRINK (HARUS SAMA)
for i in range(H):
    total_fc.iloc[i] *= BEST["shrink"]**i
    freq_fc.iloc[i]  *= BEST["shrink"]**i

sev_fc = np.where(freq_fc > 0, total_fc / freq_fc, 0)

# ============================================================
# BUILD SUBMISSION
# ============================================================

predictions = {}

for i, period in enumerate(future_periods):

    key_total = f"{period.year}_{str(period.month).zfill(2)}_Total_Claim"
    key_freq  = f"{period.year}_{str(period.month).zfill(2)}_Claim_Frequency"
    key_sev   = f"{period.year}_{str(period.month).zfill(2)}_Claim_Severity"

    predictions[key_total] = max(total_fc.iloc[i], 0)
    predictions[key_freq]  = max(freq_fc.iloc[i], 0)
    predictions[key_sev]   = max(sev_fc[i], 0)

submission = sample_sub.copy()
submission["value"] = submission["id"].map(predictions)
submission["value"] = submission["value"].fillna(0)

submission = submission[["id","value"]]
submission.to_csv("submission.csv", index=False)

print("Submission created — USING STAGE 4 OPTIMIZED PARAMS")


Submission created — USING STAGE 4 OPTIMIZED PARAMS


In [7]:
print(submission.head(9)) ## belum upgrade 

                        id         value
0  2025_08_Claim_Frequency  2.328617e+02
1   2025_08_Claim_Severity  5.349794e+07
2      2025_08_Total_Claim  1.245762e+10
3  2025_09_Claim_Frequency  2.252773e+02
4   2025_09_Claim_Severity  5.358887e+07
5      2025_09_Total_Claim  1.207235e+10
6  2025_10_Claim_Frequency  2.179630e+02
7   2025_10_Claim_Severity  5.368247e+07
8      2025_10_Total_Claim  1.170079e+10


In [8]:
import pandas as pd
import numpy as np

# =====================================
# BUILD BOTH AXIS
# =====================================

df["service_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")
df["payment_month"] = df["tanggal_pembayaran_klaim"].dt.to_period("M")

service_monthly = (
    df.groupby("service_month")
      .agg(freq=("claim_id","count"),
           total=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
      .sort_values("service_month")
)

payment_monthly = (
    df.groupby("payment_month")
      .agg(freq=("claim_id","count"),
           total=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
      .sort_values("payment_month")
)

print("===== BASIC INFO =====")
print("Service months:", service_monthly["service_month"].min(), "→", service_monthly["service_month"].max())
print("Payment months:", payment_monthly["payment_month"].min(), "→", payment_monthly["payment_month"].max())
print()

print("Total months (Service):", len(service_monthly))
print("Total months (Payment):", len(payment_monthly))
print()

# =====================================
# TAIL CHECK
# =====================================

print("===== LAST 6 MONTHS (SERVICE) =====")
print(service_monthly.tail(6))
print()

print("===== LAST 6 MONTHS (PAYMENT) =====")
print(payment_monthly.tail(6))
print()

# =====================================
# VOLATILITY CHECK
# =====================================

print("===== VOLATILITY =====")
print("Service freq std/mean:", 
      service_monthly["freq"].std() / service_monthly["freq"].mean())

print("Payment freq std/mean:", 
      payment_monthly["freq"].std() / payment_monthly["freq"].mean())
print()

# =====================================
# SEASONAL CHECK (AVG PER MONTH)
# =====================================

payment_monthly["month"] = payment_monthly["payment_month"].dt.month

seasonal = payment_monthly.groupby("month")["total"].mean()

print("===== AVG TOTAL PER CALENDAR MONTH =====")
print(seasonal.sort_values())


===== BASIC INFO =====
Service months: 2024-01 → 2025-07
Payment months: 2024-01 → 2025-12

Total months (Service): 19
Total months (Payment): 24

===== LAST 6 MONTHS (SERVICE) =====
   service_month  freq         total
13       2025-02   246  1.748054e+10
14       2025-03   230  1.367924e+10
15       2025-04   207  1.090143e+10
16       2025-05   234  1.190261e+10
17       2025-06   227  1.231230e+10
18       2025-07   260  1.322321e+10

===== LAST 6 MONTHS (PAYMENT) =====
   payment_month  freq         total
18       2025-07   272  1.862361e+10
19       2025-08   245  1.546896e+10
20       2025-09   197  1.041073e+10
21       2025-10    58  4.900102e+09
22       2025-11     3  1.356322e+08
23       2025-12     2  1.366003e+08

===== VOLATILITY =====
Service freq std/mean: 0.10975628930790748
Payment freq std/mean: 0.5146225591902012

===== AVG TOTAL PER CALENDAR MONTH =====
month
2     6.121878e+09
12    7.117807e+09
10    8.021042e+09
4     8.410073e+09
1     8.550421e+09
11    8.76